In [323]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
from scipy.special import softmax


In [324]:
# Load the dataset of intents
df=pd.read_csv('dataset_full_noOR.csv')

In [325]:
sentences = list(df.text.astype(str))
labels = list(df.label.astype(str))

In [326]:
# Tokenize input sentences
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Convert text labels to numerical labels
label_map = {label: i for i, label in enumerate(set(labels))}
numerical_labels = [label_map[label] for label in labels]

# Split data into train and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(encoded_inputs['input_ids'],
                                                                      numerical_labels,
                                                                      test_size=0.2,
                                                                      random_state=42)

# Convert data to PyTorch tensors
train_inputs_tensor = torch.tensor(train_inputs)
val_inputs_tensor = torch.tensor(val_inputs)
train_labels_tensor = torch.tensor(train_labels)
val_labels_tensor = torch.tensor(val_labels)

# Create attention masks
train_attention_masks = torch.tensor([[int(token_id != tokenizer.pad_token_id) for token_id in input_ids] for input_ids in train_inputs])
val_attention_masks = torch.tensor([[int(token_id != tokenizer.pad_token_id) for token_id in input_ids] for input_ids in val_inputs])

# Create TensorDatasets
train_dataset = TensorDataset(train_inputs_tensor, train_attention_masks, train_labels_tensor)
val_dataset = TensorDataset(val_inputs_tensor, val_attention_masks, val_labels_tensor)

# Define batch size and create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))


<ipython-input-326-7eafa03295be>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs_tensor = torch.tensor(train_inputs)
<ipython-input-326-7eafa03295be>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_inputs_tensor = torch.tensor(val_inputs)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [327]:
# Define optimizer
#optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Training loop with progress bar
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    val_predictions = []
    val_true_labels = []
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}"):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        # Evaluate on validation set during training
        model.eval()
        with torch.no_grad():
            logits = model(input_ids, attention_mask=attention_mask).logits
            predictions = np.argmax(logits.detach().numpy(), axis=1)
            val_predictions.extend(predictions)
            val_true_labels.extend(labels.numpy())

    # Calculate evaluation metrics
    accuracy = accuracy_score(val_true_labels, val_predictions)
    precision = precision_score(val_true_labels, val_predictions, average='macro')
    recall = recall_score(val_true_labels, val_predictions, average='macro')
    f1 = f1_score(val_true_labels, val_predictions, average='macro')

    print(f"Epoch {epoch+1}:")
    print(f"  Training loss: {train_loss / len(train_dataloader):.4f}")
    print(f"  Validation Accuracy: {accuracy:.4f}")
    print(f"  Validation Precision: {precision:.4f}")
    print(f"  Validation Recall: {recall:.4f}")
    print(f"  Validation F1-score: {f1:.4f}")



# Evaluation
model.eval()
val_predictions = []
val_true_labels = []
for batch in val_dataloader:
    input_ids, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = np.argmax(logits.detach().numpy(), axis=1)
    val_predictions.extend(predictions)
    val_true_labels.extend(labels.numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(val_true_labels, val_predictions)
precision = precision_score(val_true_labels, val_predictions, average='macro')
recall = recall_score(val_true_labels, val_predictions, average='macro')
f1 = f1_score(val_true_labels, val_predictions, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Epoch 1: 100%|██████████| 80/80 [00:39<00:00,  2.04it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1:
  Training loss: 3.2602
  Validation Accuracy: 0.4591
  Validation Precision: 0.8035
  Validation Recall: 0.4353
  Validation F1-score: 0.5127


Epoch 2: 100%|██████████| 80/80 [00:38<00:00,  2.07it/s]


Epoch 2:
  Training loss: 1.1889
  Validation Accuracy: 0.9827
  Validation Precision: 0.9790
  Validation Recall: 0.9793
  Validation F1-score: 0.9769


Epoch 3: 100%|██████████| 80/80 [00:38<00:00,  2.08it/s]


...

Epoch 9: 100%|██████████| 80/80 [00:38<00:00,  2.08it/s]


Epoch 9:
  Training loss: 0.0506
  Validation Accuracy: 0.9961
  Validation Precision: 0.9957
  Validation Recall: 0.9955
  Validation F1-score: 0.9954


Epoch 10: 100%|██████████| 80/80 [00:38<00:00,  2.09it/s]


Epoch 10:
  Training loss: 0.0432
  Validation Accuracy: 0.9969
  Validation Precision: 0.9965
  Validation Recall: 0.9964
  Validation F1-score: 0.9963
Accuracy: 0.9749216300940439
Precision: 0.9677276746242263
Recall: 0.981896551724138
F1-score: 0.9725161710642655


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [328]:
# Sample input text
input_text = "Please compile the sales results for each day and location over the last seven weeks."

# Tokenize input text
input_encoding = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')

# Perform prediction
model.eval()
with torch.no_grad():
    input_ids = input_encoding['input_ids']
    attention_mask = input_encoding['attention_mask']
    logits = model(input_ids, attention_mask=attention_mask).logits
    probabilities = torch.softmax(logits, dim=1)
    top_k_probabilities, top_k_indices = torch.topk(probabilities, k=3, dim=1)

# Convert top k predicted labels back to text labels
top_k_predictions = [[label_map_inverse[idx.item()] for idx in top_k_indices_row] for top_k_indices_row in top_k_indices]

# Convert tensor to list for easier manipulation
top_k_probabilities = top_k_probabilities.numpy().tolist()

# Display top 3 predictions and their confidence levels
for i, (preds, probs) in enumerate(zip(top_k_predictions, top_k_probabilities), 1):
    print(f"Top {i} Predictions:")
    for pred, prob in zip(preds, probs):
        print(f"  Label: {pred}, Confidence Level (%): {prob * 100:.2f}")
    print()

Top 1 Predictions:
  Label: Request for Daily Sales Breakdown by Location - Time Horizon Specific, Confidence Level (%): 97.23
  Label: Request for Sales Forecast by Location, Confidence Level (%): 0.12
  Label: Sales Summary - Top 10 Employees - Total Sales by Store - Current Week, Confidence Level (%): 0.10



In [329]:
# Sample input text
input_text = "Please compile the sales forecast for the next 3 weeks for product hello_kitty."

# Tokenize input text
input_encoding = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')

# Perform prediction
model.eval()
with torch.no_grad():
    input_ids = input_encoding['input_ids']
    attention_mask = input_encoding['attention_mask']
    logits = model(input_ids, attention_mask=attention_mask).logits
    probabilities = torch.softmax(logits, dim=1)
    top_k_probabilities, top_k_indices = torch.topk(probabilities, k=3, dim=1)

# Convert top k predicted labels back to text labels
top_k_predictions = [[label_map_inverse[idx.item()] for idx in top_k_indices_row] for top_k_indices_row in top_k_indices]

# Convert tensor to list for easier manipulation
top_k_probabilities = top_k_probabilities.numpy().tolist()

# Display top 3 predictions and their confidence levels
for i, (preds, probs) in enumerate(zip(top_k_predictions, top_k_probabilities), 1):
    print(f"Top {i} Predictions:")
    for pred, prob in zip(preds, probs):
        print(f"  Label: {pred}, Confidence Level (%): {prob * 100:.2f}")
    print()

Top 1 Predictions:
  Label: Request for Daily Sales Breakdown by Location - Time Horizon Specific, Confidence Level (%): 15.49
  Label: Request for Sales Forecast by Location, Confidence Level (%): 7.42
  Label: Request for Sales Breakdown by Product Category, Confidence Level (%): 5.74



In [330]:
# Sample input text
input_text = "Once upon a time there was light in my life But now there's only love in the dark Nothing I can say A total eclipse of the heart"

# Tokenize input text
input_encoding = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')

# Perform prediction
model.eval()
with torch.no_grad():
    input_ids = input_encoding['input_ids']
    attention_mask = input_encoding['attention_mask']
    logits = model(input_ids, attention_mask=attention_mask).logits
    probabilities = torch.softmax(logits, dim=1)
    top_k_probabilities, top_k_indices = torch.topk(probabilities, k=3, dim=1)

# Convert top k predicted labels back to text labels
top_k_predictions = [[label_map_inverse[idx.item()] for idx in top_k_indices_row] for top_k_indices_row in top_k_indices]

# Convert tensor to list for easier manipulation
top_k_probabilities = top_k_probabilities.numpy().tolist()

# Display top 3 predictions and their confidence levels
for i, (preds, probs) in enumerate(zip(top_k_predictions, top_k_probabilities), 1):
    print(f"Top {i} Predictions:")
    for pred, prob in zip(preds, probs):
        print(f"  Label: {pred}, Confidence Level (%): {prob * 100:.2f}")
    print()

Top 1 Predictions:
  Label: Unrelated_Random_Sentence, Confidence Level (%): 75.43
  Label: Sales Report - Product Specific - Yesterday, Confidence Level (%): 4.24
  Label: Sales Report - Product Category Specific - Yesterday, Confidence Level (%): 3.21



In [331]:
# Sample input text
input_text = "A company makes raincoats and umbrellas with images of Latin alphabet on them using a printing machine. Due to the popularity of Latin alphabet, the company must make at least 1200 raincoats or umbrellas, in any combination, per week. Also, in one week, the printing machine must be kept running for at least 70 hours. A raincoat takes 0.3 hours of printing time and costs $7. An umbrella takes 0.5 hours of printing time and costs $12. Formulate this problem so as to minimize total production costs."
# Tokenize input text
input_encoding = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')

# Perform prediction
model.eval()
with torch.no_grad():
    input_ids = input_encoding['input_ids']
    attention_mask = input_encoding['attention_mask']
    logits = model(input_ids, attention_mask=attention_mask).logits
    probabilities = torch.softmax(logits, dim=1)
    top_k_probabilities, top_k_indices = torch.topk(probabilities, k=3, dim=1)

# Convert top k predicted labels back to text labels
top_k_predictions = [[label_map_inverse[idx.item()] for idx in top_k_indices_row] for top_k_indices_row in top_k_indices]

# Convert tensor to list for easier manipulation
top_k_probabilities = top_k_probabilities.numpy().tolist()

# Display top 3 predictions and their confidence levels
for i, (preds, probs) in enumerate(zip(top_k_predictions, top_k_probabilities), 1):
    print(f"Top {i} Predictions:")
    for pred, prob in zip(preds, probs):
        print(f"  Label: {pred}, Confidence Level (%): {prob * 100:.2f}")
    print()

Top 1 Predictions:
  Label: Unrelated_Random_Sentence, Confidence Level (%): 32.64
  Label: Inventory Inquiry - Product Specific Attributes, Confidence Level (%): 5.54
  Label: Request for Sales Breakdown by Hour, Confidence Level (%): 4.73



In [332]:
# Sample input text
input_text = "forecast for product x for the next 2 semesters"
# Tokenize input text
input_encoding = tokenizer(input_text, padding=True, truncation=True, return_tensors='pt')

# Perform prediction
model.eval()
with torch.no_grad():
    input_ids = input_encoding['input_ids']
    attention_mask = input_encoding['attention_mask']
    logits = model(input_ids, attention_mask=attention_mask).logits
    probabilities = torch.softmax(logits, dim=1)
    top_k_probabilities, top_k_indices = torch.topk(probabilities, k=3, dim=1)

# Convert top k predicted labels back to text labels
top_k_predictions = [[label_map_inverse[idx.item()] for idx in top_k_indices_row] for top_k_indices_row in top_k_indices]

# Convert tensor to list for easier manipulation
top_k_probabilities = top_k_probabilities.numpy().tolist()

# Display top 3 predictions and their confidence levels
for i, (preds, probs) in enumerate(zip(top_k_predictions, top_k_probabilities), 1):
    print(f"Top {i} Predictions:")
    for pred, prob in zip(preds, probs):
        print(f"  Label: {pred}, Confidence Level (%): {prob * 100:.2f}")
    print()

Top 1 Predictions:
  Label: Sales Forecast Inquiry - Product Specific - Time Period Specific, Confidence Level (%): 23.22
  Label: Request for Sales Breakdown by Product Category, Confidence Level (%): 9.00
  Label: Request for Daily Sales Breakdown by Location - Time Horizon Specific, Confidence Level (%): 4.01

